In [5]:
import os

from dotenv import load_dotenv
load_dotenv()

api_key = os.getenv("OPENWEATHER_API_KEY", "")

In [7]:
import requests

base_url = "http://api.openweathermap.org/geo/1.0/direct"

city = "Bangalore"
params = {
    'q': city,
    'appid': api_key,
}
try:
    response = requests.get(base_url, params=params)
    response.raise_for_status()
    data = response.json()
    lat = data[0]["lat"]
    lon = data[0]["lon"]
    print(lat)
    print(lon)
except Exception as e:
    print(e)

12.9767936
77.590082


In [2]:
from datetime import datetime, timedelta

target_date = datetime(year=2025, month=12, day=6)
target_time = timedelta(days=10, hours=17, minutes=30)

target_date_time = target_date + target_time
print(target_date_time)
unix_timestamp = str(int(target_date_time.timestamp()))

2025-12-16 17:30:00


In [3]:
print(unix_timestamp)

1765886400


In [ ]:
import requests 

lat = "12.9767936"
long = "77.590082"

days_ahead = (target_date_time - datetime.now()).days
params = {
    "lat": lat,
    "lon": long,
    "appid": api_key,
    "units": "metric"
}

base_url = "https://api.openweathermap.org/data/2.5/forecast"
weather_conditions = ""

try:
    response = requests.get(url=base_url, params=params)
    response.raise_for_status()
    data = response.json()
    forecasts = data['list']
    closest_forecast = None
    min_diff = float('inf')
    for forecast in forecasts:
        forecast_time = datetime.fromtimestamp(forecast['dt'])
        time_diff = abs((forecast_time - target_date_time).total_seconds())
        
        if time_diff < min_diff:
            min_diff = time_diff
            closest_forecast = forecast
    if closest_forecast:
        forecast_dt = datetime.fromtimestamp(closest_forecast['dt'])
        temp_unit = "°C"
        speed_unit = "m/s"
        
        print(f"\n{'='*50}")
        print(f"Weather Forecast")
        print(f"Location: {lat}, {lon}")
        print(f"{'='*50}")
        print(f"Requested: {target_date_time.strftime('%Y-%m-%d %H:%M')}")
        print(f"Forecast:  {forecast_dt.strftime('%Y-%m-%d %H:%M')}")
        print(f"{'-'*50}")
        # Record the weather conditions
        weather_conditions += f"Condition: {closest_forecast['weather'][0]['description'].title()}\n"
        weather_conditions += f"Temperature: {closest_forecast['main']['temp']}{temp_unit}\n"
        weather_conditions += f"Feels Like: {closest_forecast['main']['feels_like']}{temp_unit}\n"
        weather_conditions += f"Min/Max: {closest_forecast['main']['temp_min']}/{closest_forecast['main']['temp_max']}{temp_unit}\n"
        weather_conditions += f"Humidity: {closest_forecast['main']['humidity']}%\n"
        weather_conditions += f"Wind Speed: {closest_forecast['wind']['speed']} {speed_unit}\n"
        weather_conditions += f"Clouds: {closest_forecast['clouds']['all']}%\n"
        if 'rain' in closest_forecast:
            weather_conditions += f"Rain (3h): {closest_forecast['rain'].get('3h', 0)} mm\n"
        if 'snow' in closest_forecast:
            weather_conditions += f"Snow (3h): {closest_forecast['snow'].get('3h', 0)} mm\n"
        print(weather_conditions)
        print(f"{'='*50}\n")

except Exception as e:
    print(e)

{
  "cod": "200",
  "message": 0,
  "cnt": 40,
  "list": [
    {
      "dt": 1764871200,
      "main": {
        "temp": 20.4,
        "feels_like": 20.85,
        "temp_min": 19.77,
        "temp_max": 20.4,
        "pressure": 1016,
        "sea_level": 1016,
        "grnd_level": 918,
        "humidity": 90,
        "temp_kf": 0.63
      },
      "weather": [
        {
          "id": 803,
          "main": "Clouds",
          "description": "broken clouds",
          "icon": "04n"
        }
      ],
      "clouds": {
        "all": 60
      },
      "wind": {
        "speed": 5.13,
        "deg": 87,
        "gust": 9.28
      },
      "visibility": 10000,
      "pop": 0,
      "sys": {
        "pod": "n"
      },
      "dt_txt": "2025-12-04 18:00:00"
    },
    {
      "dt": 1764882000,
      "main": {
        "temp": 20.03,
        "feels_like": 20.36,
        "temp_min": 19.69,
        "temp_max": 20.03,
        "pressure": 1015,
        "sea_level": 1015,
        "grnd_level": 

In [9]:
print(weather_conditions)

Condition: Overcast Clouds
Temperature: 19.62°C
Feels Like: 19.26°C
Min/Max: 19.62/19.62°C
Humidity: 62%
Wind Speed: 3.69 m/s
Clouds: 100%



In [10]:
from enum import Enum
from pydantic import BaseModel, Field
from langchain.chat_models import init_chat_model
from langchain_core.prompts import ChatPromptTemplate

class SeatPreference(str, Enum):
    INDOOR = "indoor"
    OUTDOOR = "outdoor"

class Seating(BaseModel):
    """Contact information for a person."""
    seat_preference: SeatPreference = Field(
        default=SeatPreference.INDOOR,
        description="Whether to sit outside and enjoy the beauty of nature or \
            to sit inside because the weather is bad"
    )

system_prompt = """
You are a world class waiter in a restaurant with a great intuition of nature based on 
weather conditions. You are supposed to output the seating preference that is best suited 
for the given climatic conditions.
"""
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("user", "{query}"),
    ]
)

llm = init_chat_model("groq:llama-3.3-70b-versatile")
structured_llm = llm.with_structured_output(Seating)
pipeline = prompt_template | structured_llm
result = await pipeline.ainvoke({
    "query": weather_conditions
})


/home/msvelan/Programming/msvelan/projects/restaurant-booking-voice-agent/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [11]:
print(result)

seat_preference=<SeatPreference.INDOOR: 'indoor'>


In [13]:
seat_preference = result.seat_preference
print(seat_preference)

SeatPreference.INDOOR
